In [3]:
import base64
import requests
import time
import pandas as pd
import os

# Claude API Key
api_key = ""

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Read CSV
questions_df = pd.read_csv("C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/VLAT Questions.csv")
questions = questions_df.to_dict('records')
responses_data = []


for i, question in enumerate(questions, start=1):
    time.sleep(5)
    try:
        print(f"\nProcessing question {i}:")
        print(question)
        
        # Path to your image
        image_path = "C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/" + str(question.get('vis', '')) + ".png"
        
        # Question text
        question_text = question.get('question: ', '')
        question_options = question.get('option:', '')
        correct_ans = str(question.get('correct', '')).strip()
        
        print(f"Processing image: {image_path}")
        print(f"Question: {question_text}")
        print(f"Options: {question_options}")
        print(f"Correct answer: {correct_ans}")
        
        # Getting the base64 string
        base64_image = encode_image(image_path)
        
        headers = {
            "Content-Type": "application/json",
            "x-api-key": api_key,
            "anthropic-version": "2023-06-01"
        }
        
        payload = {
            "model": "claude-3-7-sonnet-20250219",
            "max_tokens": 5000,
            "temperature": 0.0,
            "system": "You are an assistant, skilled in reading and interpreting visually represented data.",
            "messages": [
                {
                    "role": "user",
                    "content": 'I am about to show you a graph and ask you a multiple-choice question about that graph. \n\n' +
                    
                               'Task 1: Data Extraction and Table Creation: First, explicitly list ALL numerical values you can identify on both axes, then create a structured table using markdown syntax that includes ALL data points you identified above with appropriate column headers with units. \n \n' +
                    
                                'Task 2: Sort the data: Sort the data in descending order by the numerical values. \n \n' +

                                'Task 3: Data Verification and Error Handling: Double-check if your table matches ALL elements in the graph by comparing each value in your table with the graph and updating your table with correct values, verify the sorting is correct, and before proceeding, confirm all corrections have been made and use ONLY the corrected data for analysis. \n \n' +

                                'Task 4: Question Analysis: Using ONLY the verified data in your table, compare EACH value individually with the reference value, for "less than" comparisons mark ALL values that are even slightly below the reference, for "greater than" comparisons mark ALL values that are even slightly above the reference, and show each comparison on a new line. \n \n' +

                                'Provide your reasoning with specific references to table values. \n\n' +
                    
                                'End with: "Correct Answer: ". Just write the value, nothing else. Do not write anything after this. \n\n' +
                    
                                'Let\'s solve this step by step.' 



                 
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": question_text + " " + question_options
                        },
                        {
                            "type": "image",
                            "source": {
                                "type": "base64",
                                "media_type": "image/png",
                                "data": base64_image
                            }
                        }
                    ]
                }
            ]
        }
        
        # Add retry logic for overloaded errors
        max_retries = 3
        retry_delay = 20  # seconds
        
        for retry in range(max_retries):
            try:
                time_start = time.perf_counter()
                response_raw = requests.post(
                    "https://api.anthropic.com/v1/messages",
                    headers=headers,
                    json=payload
                )
                time_end = time.perf_counter()
                
                response = response_raw.json()
                
                # If we get an overloaded error, wait and retry
                if 'error' in response and response['error'].get('type') == 'overloaded_error':
                    if retry < max_retries - 1:  # Don't sleep on the last retry
                        print(f"\nAPI overloaded, waiting {retry_delay} seconds before retry {retry + 1}/{max_retries}")
                        time.sleep(retry_delay)
                        continue
                
                break  # If we get here, we either got a good response or a different error
                
            except Exception as e:
                print(f"Error during API call (attempt {retry + 1}/{max_retries}):", str(e))
                if retry < max_retries - 1:
                    time.sleep(retry_delay)
                    continue
                raise  # Re-raise the last exception if we've exhausted all retries
        print("\nAPI Response:", response)  # Debug print
        
        if 'error' in response:
            gpt_answer = f"{response['error']['type']}: {response['error']['message']}"
            is_correct = "N/A"
        else:
            # Extract the answer after "Correct Answer: "
            full_response = response["content"][0]["text"].strip()
            if "Correct Answer: " in full_response:
                gpt_answer = full_response.split("Correct Answer: ")[-1].strip()
                # Case-insensitive comparison after stripping whitespace
                is_correct = gpt_answer.strip().upper() == correct_ans.strip().upper()
            else:
                gpt_answer = "Error: No answer in correct format"
                is_correct = "N/A"
        
        responses_data.append([gpt_answer, time_end-time_start, is_correct])
        print(f"\nAnswer: {gpt_answer}")
        print(f"Time taken: {time_end-time_start:.2f} seconds")
        print(f"Correct? {is_correct}")
        
        # Increase delay between requests to 15 seconds
        time.sleep(max(15 - (time_end-time_start), 0))
        
    except Exception as e:
        print(f"Error processing question {i}:", str(e))
        responses_data.append([f"Error: {str(e)}", 0, "N/A"])

# Create Results directory if it doesn't exist
results_dir = "C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Results/"
os.makedirs(results_dir, exist_ok=True)

results_df = pd.DataFrame(responses_data, columns=['response', 'time', 'correct_bool'])
results_df.index = range(1, results_df.shape[0] + 1)
results_df.to_csv(results_dir + "VLAT_" + str(int(time.time())) + ".csv", index_label="id")
print("\n*** Finished ***")


Processing question 1:
{'id': 1, 'dropped': 'no', 'vis': 'VLAT_a', 'item': 'a_1', 'question: ': 'What was the price of a barrel of oil in February 2015? ', 'option:': '$57.36; $47.82; $50.24; $39.72', 'correct': '$50.24 '}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_a.png
Question: What was the price of a barrel of oil in February 2015? 
Options: $57.36; $47.82; $50.24; $39.72
Correct answer: $50.24

API Response: {'id': 'msg_01RKfvWB7bD5sMixfsuJmaCY', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': "# Task 1: Data Extraction and Table Creation\n\nLooking at the graph, I can identify the following values on the axes:\n\nY-axis (Oil Price in $ per barrel): Values range from approximately 35 to 65, with gridlines at 35, 40, 45, 50, 55, 60, and 65.\nX-axis (Month): January through December 2015\n\nData points from the graph:\n- January: approximately $48.24\n- February: approximately


Processing question 4:
{'id': 4, 'dropped': 'no', 'vis': 'VLAT_a', 'item': 'a_4', 'question: ': 'Over the course of the second half of 2015, the price of a barrel of oil was ____________.', 'option:': 'rising; falling; staying', 'correct': 'falling'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_a.png
Question: Over the course of the second half of 2015, the price of a barrel of oil was ____________.
Options: rising; falling; staying
Correct answer: falling

API Response: {'id': 'msg_011NToqRCkW8SJM3XmAo7Dsu', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': "## Task 1: Data Extraction and Table Creation\n\nNumerical values on the axes:\n- Y-axis (Oil Price): Values range from 35 to 65 dollars per barrel, with increments of 5\n- X-axis (Month): January through December 2015\n\nData points from the graph:\n\n| Month | Oil Price ($ per barrel) |\n|--------|----------------------|\n| Ja


Processing question 7:
{'id': 7, 'dropped': 'no', 'vis': 'VLAT_b', 'item': 'b_2', 'question: ': 'In which country is the average internet speed the fastest in Asia?', 'option:': 'China; Hong Kong; South Korea; Vietnam', 'correct': 'South Korea'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_b.png
Question: In which country is the average internet speed the fastest in Asia?
Options: China; Hong Kong; South Korea; Vietnam
Correct answer: South Korea

API Response: {'id': 'msg_01Rff1VmUHGiHpcH5hpzjGQd', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': '## Task 1: Data Extraction and Table Creation\n\nI can identify the following numerical values from the y-axis (Speed in Mbps):\n0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22\n\nFrom the graph, I can extract the following data points:\n\n| Country | Average Internet Speed (Mbps) |\n|---------|------------------------------|\n| Australia | 7.9


Processing question 10:
{'id': 10, 'dropped': 'no', 'vis': 'VLAT_c', 'item': 'c_1', 'question: ': 'What is the cost of peanuts in Las Vegas?', 'option:': '$12; $16.7; $23.4; $35.4', 'correct': '$12 '}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_c.png
Question: What is the cost of peanuts in Las Vegas?
Options: $12; $16.7; $23.4; $35.4
Correct answer: $12

API Response: {'id': 'msg_01MiULgxewjCduVkZpvxku4j', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': '# Task 1: Data Extraction and Table Creation\n\nLooking at the graph "Hotel Costs of Room Service" which shows stacked bar charts for different cities with costs for different items (Vodka, Soda, Peanuts, Water, and Sandwich).\n\nThe y-axis shows Cost ($) from 0 to 60 in increments of 5.\n\nFor Las Vegas, I need to identify the cost of peanuts by examining where the peanuts segment begins and ends.\n\nLet me extract all the data 


Processing question 13:
{'id': 13, 'dropped': 'yes', 'vis': 'VLAT_c', 'item': 'c_4', 'question: ': 'What is the cost range of a sandwich in the cities?', 'option:': '$0 - $24.2; $0 - $55.9; $13 - $24.2; $17 - $35.2', 'correct': '$13 - $24.2'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_c.png
Question: What is the cost range of a sandwich in the cities?
Options: $0 - $24.2; $0 - $55.9; $13 - $24.2; $17 - $35.2
Correct answer: $13 - $24.2

API Response: {'id': 'msg_01FWbzdD5tpsp2B5gkEbEQTD', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': '# Task 1: Data Extraction and Table Creation\n\nLooking at the graph "Hotel Costs of Room Service", I can identify the following:\n\nY-axis values (Cost in $): 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60\nX-axis: Cities (New York City, Las Vegas, Honolulu, Atlanta, Boston, Washington D.C., Chicago, Orlando, San Francisco, Seattle)\n\nThe grap

...


Processing question 52:
{'id': 52, 'dropped': 'no', 'vis': 'VLAT_h', 'item': 'h_6', 'question: ': 'A group of the metro systems of the world has approximately 300 stations and around a 200 km system length.', 'option:': 'True; False', 'correct': 'FALSE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_h.png
Question: A group of the metro systems of the world has approximately 300 stations and around a 200 km system length.
Options: True; False
Correct answer: FALSE

API Response: {'id': 'msg_01PeDDcmp8jdqZPFfYwP4RXX', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': '## Task 1: Data Extraction and Table Creation\n\nI\'ll identify the numerical values on both axes and create a table with all data points.\n\nX-axis (Number of Stations): Values range from 100 to 450, with visible markings at 150, 200, 250, 300, 350, 400, 450\nY-axis (Total System Length in km): Values range from 150 to 60


Processing question 54:
{'id': 54, 'dropped': 'no', 'vis': 'VLAT_h', 'item': 'h_8', 'question: ': 'The metro system in Shanghai has more ridership than the metro system in Beijing.', 'option:': 'True; False', 'correct': 'FALSE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_h.png
Question: The metro system in Shanghai has more ridership than the metro system in Beijing.
Options: True; False
Correct answer: FALSE

API Response: {'id': 'msg_01XyNXWevKNhb5QDknr4XrU2', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': '# Task 1: Data Extraction and Table Creation\n\nLooking at the graph "Metro Systems of the World", I can identify the following numerical values:\n\nX-axis (Number of Stations): Values range from approximately 100 to 450\nY-axis (Total System Length): Values range from approximately 150 to 600 km\n\nThe bubble size represents Ridership (bn per year), with reference circles 


Processing question 56:
{'id': 56, 'dropped': 'no', 'vis': 'VLAT_k', 'item': 'k_2', 'question: ': 'In which state was the unemployment rate the highest in 2015?', 'option:': 'Alaska (AK); New Mexico (NM); Florida (FL); New York (NY)', 'correct': 'New Mexico (NM)'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_k.png
Question: In which state was the unemployment rate the highest in 2015?
Options: Alaska (AK); New Mexico (NM); Florida (FL); New York (NY)
Correct answer: New Mexico (NM)

API Response: {'id': 'msg_016S2pH98iaSt6v8yGR3fvQf', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': "I'll analyze this map showing unemployment rates for states in 2015.\n\n## Task 1: Data Extraction and Table Creation\n\nThe map shows unemployment rates with the following scale values:\n- 0.0%\n- 1.1%\n- 2.3%\n- 3.4%\n- 4.6%\n- 5.7%\n- 6.9%\n\nLooking at the states mentioned in the question (AK, NM, F


Processing question 59:
{'id': 59, 'dropped': 'no', 'vis': 'VLAT_l', 'item': 'l_2', 'question: ': 'For which website was the number of unique visitors the largest in 2010?', 'option:': 'Facebook; Amazon; Bing; Google', 'correct': 'Google'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_l.png
Question: For which website was the number of unique visitors the largest in 2010?
Options: Facebook; Amazon; Bing; Google
Correct answer: Google

API Response: {'id': 'msg_01P52TWJH2DaZJJXff9n46PF', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': 'I\'ll analyze this treemap visualization showing the number of unique visitors for websites in 2010.\n\n## Task 1: Data Extraction and Table Creation\n\nThe image shows a treemap visualization where the size of each rectangle represents the number of unique visitors. The websites are categorized into different groups: Financial, Computer, News, Social 


Processing question 61:
{'id': 61, 'dropped': 'no', 'vis': 'VLAT_l', 'item': 'l_4', 'question: ': 'Samsung is nested in the Financial category.', 'option:': 'True; False', 'correct': 'FALSE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_l.png
Question: Samsung is nested in the Financial category.
Options: True; False
Correct answer: FALSE

API Response: {'id': 'msg_01VkqbVrLRvSVusLS1LtA2bW', 'type': 'message', 'role': 'assistant', 'model': 'claude-3-7-sonnet-20250219', 'content': [{'type': 'text', 'text': 'Looking at the graph titled "The Number of Unique Visitors for Websites in 2010", I\'ll analyze the data and answer your question.\n\n## Task 1: Data Extraction and Table Creation\n\nThe image shows a treemap visualization of websites categorized by industry. The main categories visible are:\n- Financial (green)\n- Computer (green)\n- News (tan/beige)\n- Social Media (blue)\n- Retail (orange)\n- Search (blue)\n\nSince this is a treemap visualizat